# Day 16
## Part 1

In [24]:
from collections import namedtuple, defaultdict
from parse import parse
import itertools

Tickets = namedtuple('Tickets', 'fields my_ticket nearby_tickets')
    
    
def parse_data(s):
    fields = {}
    nearby_tickets = []
    
    parsing_others = False
    
    for line in s:
        line = line.strip()
        if line == 'your ticket:':
            pass
        elif line == 'nearby tickets:':
            parsing_others = True
        elif (p := parse('{field}: {a:d}-{b:d} or {c:d}-{d:d}', line)):
            fields[p['field']] = set(range(p['a'], p['b'] + 1)) | set(range(p['c'], p['d'] + 1))
        elif line and not parsing_others:
            my_ticket = [int(x) for x in line.split(',')]
        elif line:
            nearby_tickets.append([int(x) for x in line.split(',')])
            
    return Tickets(fields, my_ticket, nearby_tickets)


test_string = '''class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12'''.splitlines()

test_tickets = parse_data(test_string)
test_tickets

Tickets(fields={'class': {1, 2, 3, 5, 6, 7}, 'row': {6, 7, 8, 9, 10, 11, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44}, 'seat': {13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 45, 46, 47, 48, 49, 50}}, my_ticket=[7, 1, 14], nearby_tickets=[[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]])

In [21]:
def part_1(tickets):
    all_fields = set.union(*tickets.fields.values())
    return sum(
        x
        for x in itertools.chain.from_iterable(tickets.nearby_tickets)
        if x not in all_fields
    )

assert part_1(test_tickets) == 71

In [23]:
tickets = parse_data(open('input').read().strip().splitlines())
part_1(tickets)

25788

## Part 2

In [31]:
def solve(tickets):
    # First find the valid tickets
    all_fields = set.union(*tickets.fields.values())
    valid_tickets = [
        t for t in tickets.nearby_tickets
        if all(f in all_fields for f in t)
    ] + [tickets.my_ticket]
    
    # The possible fields for a column will be the fields where
    # all tickets' column are in the fields' range
    possible_fields = defaultdict(set)
    solution = {}
    n = len(valid_tickets[0])
    # Find the possible fields for the first ticket
    for i, v in enumerate(valid_tickets[0]):
        possible_fields[i] = {
            k for k in tickets.fields
            if v in tickets.fields[k]
        }
    # then do a set intersection with the rest
    # to find the possibles for all
    for t in valid_tickets[1:]:
        for i, v in enumerate(t):
            possible_fields[i] &= {
                k for k in tickets.fields
                if v in tickets.fields[k]
            }
    
    # While the solution is incomplete
    while len(solution) < n:
        for i in possible_fields:
            # Is there a column with only 1 possible field?
            # Then add that to the solution and remove it
            # from other possible fields.
            if len(possible_fields[i]) == 1:
                solution[i] = possible_fields[i].pop()
                for j in possible_fields:
                    possible_fields[j].discard(solution[i])
                    
    return {
        solution[k]: tickets.my_ticket[k]
        for k in solution
    }


solve(test_tickets)

{'class': 1, 'seat': 14, 'row': 7}

In [32]:
test_tickets_2 = parse_data('''class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
'''.splitlines())

solve(test_tickets_2)

{'row': 11, 'class': 12, 'seat': 13}

In [33]:
solve(tickets)

{'zone': 131,
 'route': 59,
 'wagon': 157,
 'row': 149,
 'duration': 71,
 'departure platform': 61,
 'departure station': 89,
 'departure date': 167,
 'departure time': 139,
 'departure location': 179,
 'departure track': 173,
 'seat': 137,
 'arrival station': 79,
 'type': 109,
 'price': 73,
 'class': 53,
 'arrival location': 151,
 'arrival platform': 83,
 'train': 67,
 'arrival track': 163}

In [34]:
import math

ticket = solve(tickets)
math.prod(
    ticket[f]
    for f in ticket
    if f.startswith('departure')
)

3902565915559